In [177]:
import pandas as pd
import seaborn as sns
import math
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

%matplotlib inline

In [4]:
test_clean = pd.read_csv('../datasets/clean_test_data.csv')

In [21]:
test_clean.head()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Kitchen Qual_Ex,Kitchen Qual_Fa,Kitchen Qual_Gd,Kitchen Qual_Po,Kitchen Qual_TA
0,2658,902301120,190,69.00,9142,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,1,0,1,0,0,0
1,2718,905108090,90,69.63,9662,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,1,0,0,0,0,0,1
2,2414,528218130,60,58.00,17104,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,1,0,0,0,1,0,0
3,1989,902207150,30,60.00,8520,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,1,0,0,0,0,1
4,625,535105100,20,69.63,9500,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,1,0,0,0,0,0,1


In [8]:
test_clean[test_clean['MS Zoning'] == 'C (all)']

In [115]:
#used this loop to remove all non residential 
nonResidential = ['C (all)','I (all)']

for each in nonResidential:
     test_clean= test_clean[test_clean['MS Zoning'] != each]
        print(each)

#had to leave this alone because it removed rows and kaggle wants all 879

C (all)
I (all)


In [13]:
test_clean.shape

(879, 96)

In [14]:
# use this loop to get the columns that had more than 80% shit missing 
moreThan80PercentGone = []
for each in test_clean.columns:
    if sum(test_clean[each].isnull())/len(test_clean) > 0.8:
        moreThan80PercentGone.append(each)
print(moreThan80PercentGone)

[]


In [185]:
#dropping the columns missing 80%
test_clean = test_clean.drop(moreThan80PercentGone, axis=1)

In [186]:
test_clean.shape 

(879, 76)

In [15]:
print(test_clean.isnull().sum().to_string()) 

Id                   0
PID                  0
MS SubClass          0
Lot Frontage         0
Lot Area             0
Street               0
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type         1
Mas Vnr Area         1
Exter Qual           0
Exter Cond           0
Bsmt Qual            0
Bsmt Cond            0
Bsmt Exposure        0
BsmtFin Type 1       0
BsmtFin SF 1         0
BsmtFin Type 2       0
BsmtFin SF 2         0
Bsmt Unf SF          0
Total Bsmt SF        0
Heating              0
Heating QC           0
Central Air          0
Electrical           0
1st Flr SF           0
2nd Flr SF           0
Low Qual Fi

In [16]:
# def object_vcs_and_nulls(df):
#     for i in df:
#         if df[i].dtype == 'object':
#             if df[i].isnull().sum() > 0:
#                 print(df[i].value_counts())  
#                 print("Number of Null Values: " + str(df[i].isnull().sum()))
#                 print("Percentage of Nulls = " + str(np.round((df[i].isnull().sum() / 14.60), 2)) + "%")
#                 print("\n")
      
# object_vcs_and_nulls(test_clean)

In [188]:
#fix Lot Frontage
mean_lot_frontage = round(test_clean['Lot Frontage'].mean(),4)
mean_lot_frontage

test_clean['Lot Frontage'].replace(np.nan, np.mean(mean_lot_frontage), inplace=True)


In [17]:
test_clean.shape

(879, 96)

In [104]:
#drop mas vnr type and area, only 1 null found in each, this one drops rows 

# test_clean.dropna(subset=['Mas Vnr Type'], inplace=True)
# test_clean.dropna(subset=['Mas Vnr Area'], inplace=True)

In [191]:
# fix Bsmt Qual, Bsmt Cond, Bsmt Exposure, BsmtFin Type 1, & BsmtFin Type 2  

test_clean['Bsmt Qual'].fillna('None', inplace=True)
test_clean['Bsmt Cond'].fillna('None', inplace=True)
test_clean['BsmtFin Type 1'].fillna('None', inplace=True)
test_clean['BsmtFin Type 2'].fillna('None', inplace=True)
test_clean['Bsmt Exposure'].fillna('None', inplace=True)


In [18]:
test_clean.shape

(879, 96)

In [193]:
#remove nulls from fireplace QU
test_clean['Fireplace Qu'].fillna('None', inplace=True)

In [194]:
# drop rows where the column 'Electrical' is null... this one drops rows 
# test_clean.dropna(subset=['Electrical'], inplace=False )
test_clean['Electrical'].fillna('SBrkr', inplace=True)

In [195]:
test_clean.shape

(879, 76)

In [196]:
# cleaning up garage nulls Garage Type -42,,
#Garage Finish-43,,Garage Qual -43,Garage Cond-43
test_clean['Garage Type'].fillna('None', inplace=True)
test_clean['Garage Finish'].fillna('None', inplace=True)
test_clean['Garage Qual'].fillna('None',inplace=True)
test_clean['Garage Cond'].fillna('None',inplace=True)

In [197]:
# I have to drop all then NaN here: Garage Yr Blt-43... cant use the first line bc it drops rows. 
#test_clean.dropna(subset=['Garage Yr Blt'], inplace=True)

test_clean.drop(columns=['Garage Yr Blt'], inplace=True)

In [210]:
test_clean['totalLivingSF'] = test_clean['Gr Liv Area'] + test_clean['Total Bsmt SF'] + test_clean['Screen Porch'] - test_clean['Low Qual Fin SF']

In [211]:
test_clean.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,...,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,totalLivingSF
0,2658,902301120,190,RM,69.00,9142,Pave,Reg,Lvl,AllPub,...,60,112,0,0,0,0,4,2006,WD,2948
1,2718,905108090,90,RL,69.63,9662,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,0,8,2006,WD,3934
2,2414,528218130,60,RL,58.00,17104,Pave,IR1,Lvl,AllPub,...,24,0,0,0,0,0,9,2006,New,2150
3,1989,902207150,30,RM,60.00,8520,Pave,Reg,Lvl,AllPub,...,0,184,0,0,0,0,7,2007,WD,1936
4,625,535105100,20,RL,69.63,9500,Pave,IR1,Lvl,AllPub,...,76,0,0,185,0,0,7,2009,WD,2973


In [215]:
test_clean.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,totalLivingSF,All Baths
0,2658,902301120,190,RM,69.00,9142,Pave,Reg,Lvl,AllPub,...,112,0,0,0,0,4,2006,WD,2948,0
1,2718,905108090,90,RL,69.63,9662,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,8,2006,WD,3934,0
2,2414,528218130,60,RL,58.00,17104,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,9,2006,New,2150,2
3,1989,902207150,30,RM,60.00,8520,Pave,Reg,Lvl,AllPub,...,184,0,0,0,0,7,2007,WD,1936,0
4,625,535105100,20,RL,69.63,9500,Pave,IR1,Lvl,AllPub,...,0,0,185,0,0,7,2009,WD,2973,1


In [214]:
test_clean['All Baths'] = test_clean['Bsmt Full Bath'] * test_clean['Full Bath'] * test_clean['Half Bath']

In [219]:
test_clean = pd.get_dummies(test_clean, columns=['Fireplace Qu', 'Foundation','MS Zoning','Kitchen Qual'], drop_first=False)

In [22]:
test_clean.to_csv('../datasets/clean_test_data.csv',index=False)